<a href="https://colab.research.google.com/github/valentinocc/Keras_cifar10/blob/master/keras_rnn_cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install tensorflow==1.13.1

In [0]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import os
from datetime import datetime
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, Activation, add, MaxPooling2D, Input, Dense, Flatten
from tensorflow.keras.models import Model

%load_ext tensorboard.notebook

In [0]:
def residual_block(x, filter_amount, project=False):  

  if (project):#project x_input to the new dimensions
    x_input = MaxPooling2D(pool_size = (2, 2), strides = (2, 2))(x)
    x_input = Conv2D(filter_amount, (1, 1), strides = (1, 1), padding = 'same')(x_input)
    x = conv_elu_block(x, filter_amount, strides = (2, 2))
    
  else:
    x_input = x
    x = conv_elu_block(x, filter_amount)
  
  x = conv_elu_block(x, filter_amount)
  
  residual_block = add([x, x_input])
  
  
  return residual_block
  
  
  
def conv_elu_block(x, filter_amount, strides = (1,1)):
  
  x = Conv2D(filter_amount, (3, 3), strides = strides, padding = 'same')(x)
  x = Activation('elu')(x)
  
  return x


def residual_neural_network(image_input, stack_amount, class_amount):
  
  x = Conv2D(16, (3, 3), strides = (1, 1), padding = 'same')(image_input)
  
  for i in range(stack_amount):
    x = residual_block(x, 16)
  
  x = residual_block(x, 32, project=True)
  
  for i in range(stack_amount):
    x = residual_block(x, 32)
  
  x = residual_block(x, 64, project=True)
  
  for i in range(stack_amount):
    x = residual_block(x, 64)
    
  x = Activation('elu')(x)
  x = MaxPooling2D(pool_size = (2, 2), strides = (2, 2))(x)
  x = Flatten()(x)
  x = Dense(class_amount, activation = 'softmax')(x)
  
  return x

def configure_augmented_data_generator(x_train):
  data_generator = ImageDataGenerator(rotation_range = 5, width_shift_range = 0.15, height_shift_range = 0.15, zoom_range = 0.13, horizontal_flip = True)
  data_generator.fit(x_train)
  
  return data_generator

In [0]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.006*8, shuffle= True)

In [0]:
INPUT_SHAPE = x_train.shape[1:]
STACK_AMOUNT = 9
CLASS_AMOUNT = 10

image_input = Input(shape=INPUT_SHAPE)
output = residual_neural_network(image_input, STACK_AMOUNT, CLASS_AMOUNT)
ResNet = Model(image_input, output)

In [0]:
TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']
tpu_model = tf.contrib.tpu.keras_to_tpu_model(ResNet, strategy = tf.contrib.tpu.TPUDistributionStrategy(tf.contrib.cluster_resolver.TPUClusterResolver(tpu = TPU_WORKER)))
tpu_model.compile( optimizer = tf.train.AdamOptimizer(learning_rate = 0.002), loss = tf.keras.losses.sparse_categorical_crossentropy, metrics = ['sparse_categorical_accuracy'])


In [0]:
EPOCHS = 5

data_generator = configure_augmented_data_generator(x_train)
tpu_model.fit_generator(data_generator.flow(x_train, y_train, batch_size = 32*8), steps_per_epoch = len(x_train) / (32*8), epochs=EPOCHS, validation_data = (x_valid, y_valid))
tpu_model.evaluate(x_test, y_test, batch_size = 16*8)